In [1]:
import pandas as pd
import numpy as np

import matplotlib as plt
matplotlib inline%
import seaborn as sns
import plotly.express as px 
import plotly.graph_objects as go

from scipy import stats as st
import math as mth

from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import r2_score

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', '{:.2f}'.format)

SyntaxError: invalid syntax (<ipython-input-1-efc3c531c629>, line 5)